In [1]:
from tensorflow.keras.datasets.mnist import load_data
import tensorflow as tf
import numpy as np


(X_train, y_train), (X_test, y_test) = load_data()

In [2]:
# X_train = X_train / 255.
# X_test = X_test / 255.

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)


X_train = X_train.reshape([*X_train.shape, 1])
X_test = X_test.reshape([*X_test.shape, 1])

In [3]:
learning_rate = 0.001
training_iters = 1000
batch_size = 128
display_step = 10

In [4]:
n_input = 784 # MNIST data input (shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

In [5]:
# Dropout
# Aplicamos o dropout para reduzir o overfitting. O dropout vai eliminar algumas unidades (nas camadas ocultas, de entrada e de saída) na rede neural.
# A decisão sobre qual neurônio será eliminado é randômica e aplicamos uma probabilidade para isso. Esse parâmetro pode ser ajustado para otimizar o desempenho da rede.
dropout = 0.75 # Dropout, probabilidade para manter unidades

In [6]:
print(X_train.shape)
print(y_train.shape)

(60000, 28, 28, 1)
(60000,)


In [7]:
# Variáveis para os pesos e bias

# Pesos
# Cada neurônio da camada oculta é conectado a um pequeno grupo de tensores de entrada (input) de dimensão 5x5. Com isso, a camada oculta terá um tamanho de 24x24.
wc1 = tf.Variable(tf.random.normal([5, 5, 1, 32])) # 5x5 conv, 1 input, 32 outputs
wc2 = tf.Variable(tf.random.normal([5, 5, 32, 64])) # 5x5 conv, 32 inputs, 64 outputs
wd1 = tf.Variable(tf.random.normal([4*4*64, 1024])) # fully connected, 7*7*64 inputs, 1024 outputs
wout = tf.Variable(tf.random.normal([1024, n_classes])) # 1024 inputs, 10 outputs (class prediction)

# Bias
bc1 = tf.Variable(tf.random.normal([32]))
bc2 = tf.Variable(tf.random.normal([64]))
bd1 = tf.Variable(tf.random.normal([1024]))
bout = tf.Variable(tf.random.normal([n_classes]))

In [8]:
def conv2d(image, w, b):
    conv = tf.nn.conv2d(image, filters=w, strides=[1,1,1,1], padding='VALID')
    conv = tf.nn.bias_add(conv, b)
    return tf.nn.relu(conv)

In [9]:
def max_pool(image, k):
    return tf.nn.max_pool(image, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='VALID')

In [10]:
def full_connected(input, w, b):
    return tf.add(tf.matmul(input, w), b)

In [24]:
def cross_entropy(y_true, y_pred):
    y_true = tf.one_hot(y_true, n_classes)
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred))

In [20]:
# Cria o otimizador usando Adam
optimizer = tf.optimizers.Adam(learning_rate)

In [21]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
dataset = dataset.repeat().shuffle(X_train.shape[0]).batch(batch_size)

In [27]:
for i, (batch_X_train, batch_y_train) in enumerate(dataset.take(1)):
    conv1 = conv2d(batch_X_train, wc1, bc1)
    conv1 = max_pool(conv1, k=2)
    conv1 = tf.nn.dropout(conv1, dropout)

    conv2 = conv2d(conv1, wc2, bc2)
    conv2 = max_pool(conv2, k=2)
    conv2 = tf.nn.dropout(conv2, dropout)

    dense1 = tf.reshape(conv2, [-1, wd1.shape[0]])
    dense1 = full_connected(dense1, wd1, bd1)
    dense1 = tf.nn.relu(dense1)

    pred = full_connected(dense1, wout, bout)
    loss = cross_entropy(batch_y_train, pred)
    
    # optimizer.minimize(loss, [wout, bout])

    
    # loss_val = loss(dense1, batch_y_train, wout, bout, n_classes)
    # print(loss_val)
    # optimizer.minimize(loss_val, [wout, bout])
    # with tf.GradientTape() as t:
    #     gradient = t.gradient(loss_val, [wout, bout, wd1, bd1, wc2, bc2, wc1, bc1])
    #     print(gradient)
        # optimizer.apply_gradients(zip(gradient, [wout, bout, wd1, bd1, wc2, bc2, wc1, bc1]))


    # Faz uma previsão
    #y_pred = logistic_regression(batch_X_train, wout, bout)

    # Calcula o erro
    #loss = cross_entropy(batch_X_train, batch_y_train, n_classes)

    # Calcula a acurácia
    #acc = accuracy(batch_X_train, batch_y_train)

    # Print
    #print("Número do Batch: %i, Erro do Modelo: %f, Acurácia em Treino: %f" % (i, loss, acc))
    
        

TypeError: 'tensorflow.python.framework.ops.EagerTensor' object is not callable